ndarrayについて

ndarrayは、同じ属性や大きさを持った要素を持つ<br>
多次元配列を扱うためのクラスの１つということです

- 同じ型を持つ要素しか格納することができない
- 各次元ごとの(2次元なら列ごとや行ごと)の要素数は必ず一定
- C言語を元に、最適化された行列演算を行うため効率的な処理をすることができる


ndarrayにすることで、多次元データを扱うための便利な属性を使用することができる

In [1]:
import numpy as np
a = np.array([1,2,3])
print(type(a))

b = np.array([[1,2,3],[4,5,6]])
print(a)
print(b)
print(b.T) #転置
print(a.T) #転置
print(a.data) #メモリの位置を示す
print(a.dtype) #データ型

<class 'numpy.ndarray'>
[1 2 3]
[[1 2 3]
 [4 5 6]]
[[1 4]
 [2 5]
 [3 6]]
[1 2 3]
int64


メモリレイアウトについて情報を表示させる

In [2]:
print(a.flags)


  C_CONTIGUOUS : True
  F_CONTIGUOUS : True
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False



In [3]:
b.flags

  C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False

In [4]:
a.flat[1] # aを1次元配列にしたときのn番目の要素を表示

2

In [5]:
b.flat[4] #bを1次元配列にしたとき4番目に要素を表示

5

【real,imag】
複素数(complex)の要素に対して、self.realとself.imag<br>
使って実部と虚部を表示

In [6]:
import numpy as np
c = np.array([1,-2.6j,2.1+3.J, 4.-3.2j]) #複素数を要素とするndarrayインスタンスを生成

print(c.real) #実部

print(c.imag) #虚部

[ 1.  -0.   2.1  4. ]
[ 0.  -2.6  3.  -3.2]


size,itemsize,nbytes
- 要素の数を表する .size<br>
- バイトオーダーでの要素１つ１つがメモリで占める、容量を表示する .itemsize <br>
- これらの積となっており配列の要素全ての容量を表示する .nbytes


In [7]:
a.size #要素の数

3

In [8]:
b.size #環境によっては4になる

6

In [9]:
a.itemsize # バイトオーダーでの要素１つ１つの長さ

8

In [10]:
b.itemsize

8

In [11]:
c.size, c.itemsize # cの方がsizeにたいしてitemsizeが大きくなる

(4, 16)

In [12]:
a.nbytes # バイトオーダーでの配列の長さ　環境によっては12になる

24

In [13]:
b.nbytes #環境によっては24になる

48

In [14]:
c.nbytes

64

In [15]:
a.size * a.itemsize == a.nbytes #この等式が成り立つ

True

ndim,shape

次元数と形状(shape)について表示させる<br>
 - .ndim(次元数)
 - .shape(形状)

In [16]:
a.ndim #次元数

1

In [17]:
b.ndim

2

In [18]:
a.shape #形状

(3,)

In [19]:
b.shape #形状

(2, 3)

strides

.stridesです。このプロパティは各次元方向に１つ要素を移動するためにはメモリ場で<br>
何バイト動く必要があるかを示したものです

In [20]:
d = np.array([[[20,3,2],[2,2,2]],[[4,3,2],[5,7,1]]]) # 3次元配列を生成
d.shape, d.ndim #形状を次元数を表示


((2, 2, 3), 3)

In [21]:
a.strides #各次元方向(axis=ndim, axis=ndim-1,....,axis=1, axis=0)における1つの要素
#移動するためのバイトオーダーでの距離　環境によっては(4,)となる

(8,)

In [22]:
b.strides # .ndim = 2 環境によっては(12, 4)となる

(24, 8)

In [23]:
c.strides # .ndim = 3

(16,)

In [24]:
d.strides # .ndim = 3 環境によっては(24, 12, 4)となる

(48, 24, 8)

ctypes,base

.ctypesモジュールを使った操作を行うためのイテレータとなります。<br>
.baseはこの配列がビューであるならビューをしている元の配列を示す <br>
ctypes = コピー<br>
base = ビュー

In [25]:
a.ctypes.data # ctypesモジュールを使った操作

4337602864

In [26]:
a.base #aのベースとなる配列はどこか。

In [27]:
e = a[:2]
e.base

array([1, 2, 3])

In [28]:
e.base is a

True

In [29]:
a.base is e.base

False

Memory Layout<br>
NumPyにおける行列計算のパフォーマンスをより向上させるためにはメモリ上で<br>
ndarrayの要素がどのように格納されているかを知る必要があります。内部メモリーで<br>
どのように配列が保存されているかを意識しておくだけで、随分と理解が進みます。


クラスndarrayで生成されたインスタンスはメモリ上では1次元で保存されます。<br>
ここに登録される情報で、データ型や形状(shape)といった１次元で並べられた要素データ<br>
読み取り方を指定した部分をメタデータと呼ぶ

このメタデータに続き、要素をデータ化したものが並んでいきます。データの格納の仕方は大きく分けて2つ存在します。<br>

1つはローメジャー(row-major)オーダーでもう1つはカラムメジャー(column-major)オーダーです。<br>前者はC言語で使われている並べ方で、後者はフォートランやMatlabで使われている並べ方です。<br>

先ほど紹介した属性で.flagsというものがありましたが、その中に次のような部分がありました。

In [30]:
C_CONTIGUOUS: True #ローメジャーで読み込むことが可能かどうかといったもの
F_CONTIGUOUS: False #カラムメジャーで読み込むことが可能かどうかを示しています

NumPyの引数でorderというものがありますが、基本的には'C'とするとローメジャーに<br>Fとするとカラムメジャーでデータが格納されていきます。

この2つの違いはどの次元方向からデータを格納していくかにあります。ローメジャーでは高い次元から<br>(axisの番号が若い順)格納していき、カラムメジャーでは低い次元(axisの番号が高い順)から格納していきます。

2次元配列の例で詳しく解説してみます。例えば、以下のような2×3の2次元配列があったとします。<br>

2次元でいくとローメジャー(order=’C’)では行方向から順に要素が格納されていきます。<br>

一方でカラムメジャー(order=’F’)では列方向から順に要素が格納されていきます。<br>

カラムメジャー <br>

列方向は2次元においてはaxis=0、行方向はaxis=1となり、軸の番号は変わりはするものの同じ大小関係が成り立ちます。このように配列のデータはメモリ上に格納されています。<br>

ストライド

列方向に処理を展開していきたい場合、ローメジャーの方が１つ１つの要素のメモリー上における距離が小さくてすみます。一方、カラムメジャーで同じ処理を行うと行方向に収納されているので列ごとに移動するためのバイト数が大きくなり、効率的に計算を行うことができません。

この1つ1つの要素にアクセスするためにメモリー上での移動距離をバイト数で表したものをストライド(strides)と呼びます。

これはndarrayの属性(attributes)の１つとなっており、この情報を見ることで要素の距離をつかむことができます。カラムメジャーとローメジャーでそれぞれ同じ配列を格納してみます。

In [31]:
a = np.random.randn(100, 100)

In [32]:
b = np.array(a, order= 'C') #row-major

In [33]:
c = np.array(a, order= 'F') # column-major

In [34]:
b.strides, c.strides #ストライドを見ると、距離が逆転している。

((800, 8), (8, 800))

In [35]:
np.allclose(b, c) #配列の要素が全部一致しているか確認

True

スライス表記を使って要素を100個飛ばしに読み込むように指定してみて計算速度の違いを確かめてみます。こうすることで、メモリー上の値を読み込む時、隣の要素を読み込むためにジャンプする必要のあるバイト数が増えるため計算速度が低下してしまいます

In [36]:
x = np.ones((100000,))
y = np.ones((100000*100,))[::100] #100個飛ばしに読み込む
x.strides #1つ隣の要素にたどりつくために8バイト分ジャンプするだけでいい

(8,)

In [37]:
y.strides #1つ隣の要素にたどり着くために800バイト分ジャンプする必要がある

(800,)

In [38]:
x.shape, y.shape

((100000,), (100000,))

In [39]:
%timeit x.sum() #こちらの方が断然早い

32.2 µs ± 107 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [40]:
%timeit y.sum()

213 µs ± 17.1 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [41]:
y_copy = np.copy(np.ones((1000000*100,))[::100])
y_copy.strides

(8,)

In [42]:
%timeit y_copy.sum()

403 µs ± 83.3 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


ブロードキャスト
ブロードキャストは計算処理を行う際に、適宣配列を拡張してくれる便利な機能です
例えば、配列「a」の全要素に対して1を加算したいとき

In [43]:
a += 1

とするだけで全ての要素に対して１が加算されます。このときブロードキャストが適用されています(aの次元はいくつでも構いません)。2次元配列に対して1次元配列を足し合わせることも可能です。

In [44]:
a = np.array([1,2,3])
b = np.array([[1,1,1],[2,4,1]]) #2次元配列
b + a #ブロードキャスト適用

array([[2, 3, 4],
       [3, 6, 4]])